<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls "/content/gdrive/My Drive/"

In [0]:
!ls "/content/gdrive/My Drive/Inpex files "

In [0]:
pip install tika

In [0]:
# from shutil import copyfile

# copyfile("/tika-server-1.18.jar", "/tmp//tika-server.jar")

In [0]:
# os.environ["TIKA_PATH"] = "/tmp/tika-server-1.18.jar"
# os.environ["TIKA_SERVER_JAR"] = "/tmp/tika-server-1.18.jar"
# print(os.environ["TIKA_SERVER_JAR"])


In [0]:
import os
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
import tika
from tika import parser

In [0]:
# Text parsing library
# import tika
# from tika import parser

# Deal with regular expression(text processing)
import re

# Handle the excel spreadsheet
import openpyxl

import os

tika.initVM()

# Read the ;PDF file
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.pdf')

# parsed = parser.from_file('/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.pdf')
content = parsed["content"].strip()

# Read the spreadsheet for later use
# wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_100OF107_IPX Reviewed_AC.xlsx")
sheet = wb['Template']

# Starting writing the sheet beginning line#2
sheet_row_num = 2

# PDF columns
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"
             , "sys_tem"]

# Spreadsheet cells
cells = ["AL", "S", "AC", "AB", "AA", "Z", "Y", "X", "K", "J", "I", "H", "G", "F", "D"]

cols = {}

In [0]:

# page_num_printed = 5479
page_num_printed = 5962
page_num = 1

total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())
# total_pages = 5
# print(total_pages)

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))


In [9]:
# page_num_printed = 5479
page_num_printed = 5962
page_num = 1

total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())
# total_pages = 5
# print(total_pages)

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))

import traceback

'''
## Pages ##
'''
# print(page_num)
# print(total_pages)

while True:
  skip = 11
# for page_num in range(1, total_pages):
  print("Page#%i"%page_num)
  print("Page num printed#%i"%page_num_printed)
  
  start_of_page = "Page " + str(page_num_printed) +" of 13699"
  end_of_page = "Page " + str(page_num_printed + 1) +" of 13699"
  page_content = content[content.find(start_of_page): content.find(end_of_page)]
  
#   print("----- Start: Page Content of page:{} -----".format(page_num_printed))
#   print(page_content)
#   print("----- End: Page Content of page:{} -----".format(page_num_printed))
  
  ## Extract the table
  try:
    p = re.compile("(A*VE\\s+DATE)", re.IGNORECASE)
    for m in p.finditer(page_content):
      start_of_table = m.start()+8

    end_of_table = re.search("(FINAL\\s+)*INSPECTION", page_content, flags = re.I)
#     print("1-End of table:{}".format(end_of_table))
    if end_of_table is None:
      end_of_table = re.search("(VISUAL\\s+)*INSPECTION", page_content, flags = re.I)
      skip = 29
#       print("2-End of table:{}".format(end_of_table))
    
    if end_of_table is None: 
      end_of_table = re.search("\\(*REMARK", page_content, flags = re.I)
      skip = 50
#       print("3-End of table: {}".format(end_of_table))
    
    end_of_table = end_of_table.start() - skip
#     print("End of table:{0} and skip:{1}".format(end_of_table,skip))
    
    table = page_content[start_of_table:end_of_table]

    # Data cleansing
    table = table.replace("·","-")\
      .replace("•","")\
      .replace(" ' "," ")\
      .replace("l4", "14")\
      .replace(",", "")\
      .replace(" . ", " ")\
      .replace(" - ", " ")
    
    table = re.sub("OS-([A-Z]{3})", r"05-\1", table,  flags=re.I)
    table = re.sub("(\d{1,2})\.*-\.*([A-Z]{3})\.*-\.*(\d{2,4})", r"\1-\2-\3", table,  flags = re.I)
    table = re.sub("\s+(\d{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\.*-\.*([A-Z]{3})\.*-\.(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+OS\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 05-\1-\2 ", table, flags = re.I)
    table = re.sub("\s+Ol\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 01-\1-\2 ", table, flags = re.I)
    table = re.sub("\-No\>J\-", r"-Nov-", table, flags = re.I)
    table = re.sub("\.+\s*Qct", r"-Oct", table, flags = re.I)
    table = re.sub("12S|l2S|l25", r"125", table)
    table = re.sub("lOS|l0S|l05|1OS|10S", r"105", table)
    table = re.sub(" 19-Nov ", "19-Nov-14", table)
    table = re.sub("(\d\d)\-(.*)?\-(\d\d)", r"\1-\2-\3", table)
    table = re.sub("(0l!!c)|(0i!!c)", "Dec", table)
    table = re.sub("\-0ec.*?\-", "-Dec-", table)
    table = re.sub("\-Oec\-", "-Dec-", table) 
    table = re.sub("(\d\d)\-([A-Z]{3})\s+", r"\1-\2-9999", table, flags = re.I)
    table = re.sub("(\d\d)\-\s*([A-Z]{3})\s*\-\s*(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("\s*l\s*S", "15", table)
    table = re.sub("\s*l\s*O\s*", "10", table)
    table = re.sub("OB", r"08", table, flags = re.I)
    table = re.sub("f|j\s*a\s*n", "Jan", table, flags = re.I)
    table = re.sub("O!Han", "01-Jan", table, flags = re.I)
    table = re.sub("0i!!!c", "Jan", table, flags = re.I)
    table = re.sub("\d\-\d\s*\-[a-z]{3}\-14", "27-Dec-14", table, flags = re.I)
    table = re.sub("\-vi", "-14", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-([a-z]){2,}\-(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)(\.|\:|,|;)*\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)\-([a-z]{2,})(\-\d{2,})+", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\-\s*0\s*e\s*c\s*\-", "-Dec-", table, flags = re.I)
    table = re.sub("Dcc", "Dec", table, flags = re.I)
    table = re.sub("([a-z]{3})\-1", r"\1-14", table, flags = re.I)
    table = re.sub("([a-z]{3})\-144", r"\1-14", table, flags = re.I)
    table = re.sub("De", "Dec", table, flags = re.I)
    table = re.sub("Decc", "Dec", table, flags = re.I)
    table = re.sub("(\d+)\-Dec$", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\:\-", "-", table, flags = re.I)
    table = re.sub("Janan", "Jan", table, flags = re.I)
    table = re.sub("1\s*\.*S|l\.*\s*S|\]\s*\.*S", "15", table, flags = re.I)
    table = re.sub("2S", "25", table, flags = re.I)    
    table = re.sub("os\.|oa\.", "05-", table, flags = re.I)
    table = re.sub("J\.i\:n|J\s*\.*;\s*\.*m|J\s*\.*l\s*\.*n|j\s*\.*t\s*\.*n|j\s*\.*t\s*\.*m|j;*:*m|j:*;*m|jo1n|j01n|join|joln|j0ln", "Jan", table, flags = re.I)
    table = re.sub("l\s*\.*l", "11", table, flags = re.I)
    table = re.sub("l(\d)", r"1\1", table, flags = re.I)
    table = re.sub("(\d)l", r"\1(1)", table, flags = re.I)
    table = re.sub(">|<|\s*..._\s*|\-n", "", table, flags = re.I)
    table = re.sub("([a-z]{3})(\d\d)", r"\1-\2", table, flags = re.I)
    table = re.sub("\-(\d{3,})", "14", table)
    
    
    '''
    ## Rows ##
    '''
    for row in [x for x in table[2:].splitlines() if x]:
      row = row.strip()
      row = re.sub("-\\s*$", "", row)
      row = re.sub("^.*?\\s+", "", row)
      row = re.sub("^\\s*\\w+\\s+", "", row)
      row = re.sub("\s+[0-9-_]+$", "", row)
      row = re.sub("\s+", " ", row)
      row = row.strip()
            
      if len(row) > 55  and\
        hasNumbers(row):
        
        print("--> Raw row:%s"%row)
                
        # Check if the second last columna is a date columns or not
        # In some cases the last date column is not read
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
        
        print("Initially Last column is:{}".format(last_col))
        # if last column is not a date - add last and second last date columns
        if (not bool(re.match("\d\d\-[a-z]{3}\-\d\d", last_col, flags = re.I))):
          print("Both date columns missing")
          # append the available date column at the end of the row
          row = row + " 99-Jan-9999"
        
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
        print("splitting again and the last column is {0} and second last column is {1}".format(last_col,second_last_col))
        if (bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I))):
          print("second last column is a date")
                     
                     
        # Second last column is not a date but last column is
        if (not bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I)) and\
                    bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", last_col, flags = re.I))):
          print("second last column: %s"%second_last_col)
          # append the available date column at the end of the row
          row = row + " "\
            + str(int(last_col.split("-")[0]) + 1) + "-"\
            + last_col.split("-")[1] + "-"\
            + last_col.split("-")[2]
          
      

        print(row)
        print("----------------------------------")

        global r
        r = row
        col_num = 0

        for col_name in col_names:
          col_val = get_col()
          cols[col_name] = col_val
          col_val = re.sub(r"([0-9]+)_", "\g<1>",col_val)
          col_val = re.sub(r"_([0-9]+)", "\g<1>",col_val)
          col_val = re.sub(r"\s+-([0-9]+)", " \g<1>",col_val)
          col_val = re.sub(r"([0-9]+)-\s+", "\g<1> ",col_val)

          # Write in spreadsheet
          cell = cells[col_num] + str(sheet_row_num)    
          sheet[cell].value= col_val
#           print("Col Name:{0}, col value:{1} written in cell {2}".format(col_name, col_val,cell))
          col_num = col_num + 1

        r = re.sub("\\s+", "_", r)
        cols["spool_no"] = r
        cell ="D"+str(sheet_row_num)    
        sheet[cell].value = r

        cell ="C"+str(sheet_row_num)    
        sheet[cell].value = page_num

        sheet_row_num = sheet_row_num + 1
  except Exception:
    print("Exception on page%i" %page_num)
    traceback.print_exc()
    pass
  
#   print("page_num:{0}, total_pages:{1}".format(page_num, total_pages))

  if(page_num == total_pages):
    break
          
  page_num = page_num + 1
  page_num_printed = page_num_printed + 1

# print("cell d2 value {}".format(sheet["d2"].value))
# wb.save("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")    
wb.save("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.xlsx")
# wb.save("/content/gdrive/My Drive/Colab Notebooks/data/test.xlsx")


Page#1
Page num printed#5962
--> Raw row:A2EC-CJan14-15B0-01 7 108 113 105 119 117 112 203 205 218 211 236 215 25-Dec-14 27-Dec-14
Initially Last column is:27-Dec-14
splitting again and the last column is 27-Dec-14 and second last column is 25-Dec-14
second last column is a date
A2EC-CJan14-15B0-01 7 108 113 105 119 117 112 203 205 218 211 236 215 25-Dec-14 27-Dec-14
----------------------------------
--> Raw row:AlEA14 LN029-3SC0-32 7 119 114 112 101 105 110 227 222 210 217 225 220 25-Dec-14 27-Dec-14
Initially Last column is:27-Dec-14
splitting again and the last column is 27-Dec-14 and second last column is 25-Dec-14
second last column is a date
AlEA14 LN029-3SC0-32 7 119 114 112 101 105 110 227 222 210 217 225 220 25-Dec-14 27-Dec-14
----------------------------------
--> Raw row:C4EB-CJan14-15B0-01 7 103 108 104 105 113 107 241 223 217 227 218 225 15-Dec-14 27-Dec-14
Initially Last column is:27-Dec-14
splitting again and the last column is 27-Dec-14 and second last column is 15-De

Traceback (most recent call last):
  File "<ipython-input-9-14118ddb7cb3>", line 58, in <module>
    end_of_table = end_of_table.start() - skip
AttributeError: 'NoneType' object has no attribute 'start'



~033187 A1EA-WS008-15A0-05 7 117 110 102 100 118 109 203 217 208 219 207 211 07-Dec-14 12-Dec-14
----------------------------------
--> Raw row:AlEA14 CJan14-15B0V-07 7 119 116 105 110 116 113 229 234 219 246 237 233 07-Dec-14 12-Dec-14
Initially Last column is:12-Dec-14
splitting again and the last column is 12-Dec-14 and second last column is 07-Dec-14
second last column is a date
AlEA14 CJan14-15B0V-07 7 119 116 105 110 116 113 229 234 219 246 237 233 07-Dec-14 12-Dec-14
----------------------------------
--> Raw row:AlEA14 CJan14-15B0-04 7 106 104 107 108 124 110 207 217 208 216 204 210 07-Dec-14 12-Dec-14
Initially Last column is:12-Dec-14
splitting again and the last column is 12-Dec-14 and second last column is 07-Dec-14
second last column is a date
AlEA14 CJan14-15B0-04 7 106 104 107 108 124 110 207 217 208 216 204 210 07-Dec-14 12-Dec-14
----------------------------------
Page#140
Page num printed#6101
--> Raw row:5035406 Al EB-DC007 14-05 7 119 117 110 102 100 110 237 218 21

Traceback (most recent call last):
  File "<ipython-input-9-14118ddb7cb3>", line 58, in <module>
    end_of_table = end_of_table.start() - skip
AttributeError: 'NoneType' object has no attribute 'start'



Initially Last column is:17-Dec-14
splitting again and the last column is 17-Dec-14 and second last column is 231-15
second last column: 231-15
PA168-1CE1-02 1 140 114 137 100 122 123 298 237 205 214 200 231-15 17-Dec-14 18-Dec-14
----------------------------------
--> Raw row:A2EJ14 N1126-1CE1-03 1 118 124 100 134 108 117 238 225 200 237 243 229 15-Dec-14 17-Dec-14
Initially Last column is:17-Dec-14
splitting again and the last column is 17-Dec-14 and second last column is 15-Dec-14
second last column is a date
A2EJ14 N1126-1CE1-03 1 118 124 100 134 108 117 238 225 200 237 243 229 15-Dec-14 17-Dec-14
----------------------------------
--> Raw row:A2EJ-N1218-1CE1-01 1 100 121 119 104 135 116 210 200 298 253 300 252 15-Dec-14 17-Dec-14
Initially Last column is:17-Dec-14
splitting again and the last column is 17-Dec-14 and second last column is 15-Dec-14
second last column is a date
A2EJ-N1218-1CE1-01 1 100 121 119 104 135 116 210 200 298 253 300 252 15-Dec-14 17-Dec-14
----------------